In [1]:
from pre_processing import df_all, df_week_player,df_week_team
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import numpy as np
import re
import os
import datetime

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

In [3]:
df_all.head()

,Date,Player,Position,Team Name,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Season,Weekday,Week,Year,Year-Week,Load Per Minute,Distance Per Minute,IMA COD Imbalance,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Volumn Risk Score
0,2021-07-01,Player1,Central Midfielder,Team1,102.0,5162.0,491.0,3.0,0.0,7.0,2.0,96.7,0.0,25.7,50.0,39.0,2021/22,Thursday,26,2021,2021-W26,4.813725,50.607843,0.282051,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
180310,2021-07-01,Player73,Goal Keeper,Team2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Thursday,26,2021,2021-W26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
81760,2021-07-01,Player96,Winger,Team3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Thursday,26,2021,2021-W26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
48910,2021-07-01,Player68,Attacker,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Thursday,26,2021,2021-W26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
213160,2021-07-01,Player194,Central Midfielder,Team4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Thursday,26,2021,2021-W26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0


In [4]:
df_week_player.reset_index().head()

,Player,Position,Team Name,Season,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,IMA COD Imbalance,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Volumn Risk Score
0,Player1,Central Midfielder,Team1,2021/22,2021,26,2021-W26,102.0,5162.00,491.00,3.0,0.0,7.0,2.0,96.70,0.00,25.70,50.0,39.0,4.813725,50.607843,0.282051,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
1,Player1,Central Midfielder,Team1,2021/22,2021,27,2021-W27,51.2,5077.60,480.48,16.0,3.8,14.6,3.2,231.94,41.08,27.10,15.8,19.0,9.384375,99.171875,-0.168421,0.626836,0.990125,0.987033,1.951220,2.500000,1.454183,1.290323,1.538104,2.500000,1.031988,0.435022,0.612903,1.412887,1.416064,Low,Moderate,Moderate,High,High,High,Moderate,High,High,Moderate,Low,Low,High,High,2,3,2,1
2,Player1,Central Midfielder,Team1,2021/22,2021,28,2021-W28,55.4,4672.20,501.36,16.0,2.6,16.2,4.8,202.18,12.28,27.40,24.8,27.6,9.049819,84.335740,-0.101449,0.778440,0.944676,1.017744,1.413428,1.331967,1.295585,1.408451,1.179927,0.831347,1.025603,0.782038,0.931174,1.189964,1.113279,Low,Moderate,Moderate,High,High,Moderate,High,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,0,3,1,1
3,Player1,Central Midfielder,Team1,2021/22,2021,29,2021-W29,57.6,4666.34,523.60,18.6,3.6,16.0,6.0,219.10,20.46,28.24,15.8,16.0,9.090278,81.012847,-0.012500,0.876168,0.965310,1.036808,1.306914,1.378676,1.150880,1.349892,1.150435,1.200231,1.033463,0.623343,0.661594,1.108681,1.040524,Moderate,Moderate,Moderate,High,High,Moderate,High,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,0,3,2,0
4,Player1,Central Midfielder,Team1,2021/22,2021,30,2021-W30,65.6,5769.14,561.52,16.8,1.4,16.8,7.0,233.08,14.08,27.44,22.4,25.4,8.559756,87.944207,-0.118110,0.998714,1.107730,1.064261,1.100975,0.658291,1.115431,1.280438,1.123267,0.887766,1.002508,0.926833,1.029574,1.025930,1.073900,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,1,0,0


In [5]:
df_week_player["IMA COD Imbalance"].abs().head()

Player   Position            Team Name  Season   Year  Week  Year-Week
Player1  Central Midfielder  Team1      2021/22  2021  26    2021-W26     0.282051
                                                       27    2021-W27     0.168421
                                                       28    2021-W28     0.101449
                                                       29    2021-W29     0.012500
                                                       30    2021-W30     0.118110
Name: IMA COD Imbalance, dtype: float64

In [17]:
# List of metrics to plot
metrics = ['Duration', 'Total Distance(m)', 'Total Player Load', 'Acc 2m/s2 Total Effort',
           'Acc 3m/s2 Total Effort', 'Dec 2m/s2 Total Effort', 'Dec 3m/s2 Total Effort',
           'High Intensity Distance(m)', 'Sprint Distance(m)', 'Maximum Velocity(m/s)',
           'IMA COD(left)', 'IMA COD(right)']

metrics_classes = {
    "Intensity": ['Load Per Minute', 'Distance Per Minute'],
    "Agility": ['Acc 2m/s2 Total Effort','Acc 3m/s2 Total Effort', 'Dec 2m/s2 Total Effort', 'Dec 3m/s2 Total Effort'],
    "IMA": ['IMA COD(left)', 'IMA COD(right)'],
    "Volumn": ['Duration', 'Total Distance(m)', 'Total Player Load']
}
